In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy


# Import Required Libraries & Data Loading

In [2]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [3]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [6]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
spacy.load("en_core_web_sm")

In [7]:
# general Settings

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.01
BATCH_SIZE = 90
NUM_EPOCHS = 10
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 400
HIDDEN_DIM = 128
NUM_CLASSES = 2

# Text & label Preparation

In [8]:
# Define feature processing
"""
Define the fields for the data.
"""
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

AttributeError: module 'torchtext' has no attribute 'legacy'

In [ ]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [ ]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

In [ ]:
# process the dataset

fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path =  'IMDB Dataset.csv',
                    format =  'csv',
                    skip_header =  True,
                    fields =  ('review', 'sentiement')
)

# Data Split

In [ ]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

In [ ]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

# Data Observation after Tokenization

In [ ]:
# Look at first traning example

print(vars(train_data.examples[2000]))

In [ ]:
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

 2 extra value in vocabulary is because added (unknown) and (padding)

In [ ]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
most_common_words = 
print(most_common_words)

In [ ]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

# Data Preparation for Batch wise Implimentation

In [ ]:
# Define Dataloader

train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = BATCH_SIZE,
        sort_within_batch = None,
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        device = DEVICE
    )

In [ ]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

In [ ]:
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data),
        batch_size = BATCH_SIZE,
        sort_within_batch = None,
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        device = DEVICE
    )

# Model Building

In [ ]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        ### ADD YOUR CODE HERE ###
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.i2h = torch.nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.i2o = torch.nn.Linear(input_dim + hidden_dim, output_dim)
        self.softmax = torch.nn.LogSoftmax(dim=1)   
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        # text dim: [sentence length, batch size]
        text_dim = [self.input_dim, BATCH_SIZE]
        # embedded dim: [sentence length, batch size, embedding dim]
        embedded_dim = self.embedding(text)
        # output dim: [sentence length, batch size, hidden dim]
        output_dim = self.i2o(text)
        # hidden dim: [1, batch size, hidden dim]
        hidden_dim = self.i2o(text)
        # hidden dim: [batch size, hidden dim]
        output = self.i2h(output_dim)
        
        ### END YOUR CODE ###
        return output

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim = INPUT_DIM,
            embedding_dim = EMBEDDING_DIM,
            hidden_dim = HIDDEN_DIM,
            output_dim = NUM_CLASSES
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Define Accuracy

In [ ]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run

In [ ]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        predictions = model(text)
        
        ### UPDATE MODEL PARAMETERS
        loss = nn.functional.cross_entropy(predictions, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

# Model Testing

In [ ]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")